Reference: https://geodacenter.github.io/workbook/6a_local_auto/lab6a.html#local-geary

# Univariate local geary

Interestingly, following [this equation](https://www.biomedware.com/files/documentation/spacestat/Statistics/Gearys_C/Geary_s_C_statistic.htm) which explicitly calls for standardization of input data. We also do NOT divide by 2.

$$ c_i = \sum_j w_{ij} (z_i - z_j)^2 $$ 

where: 

$z_i = x_i - \bar{x}$ and $z_j = x_j - \bar{x}$, and $w_{ij}$ are the elements of the row-standardized binary symmetric spatial weight matrix W. 

or, $$ c_i = (1/m^2) * \sum_j w_{ij} (x_i - x_j)^2 $$

where,

$$ m^2 = \sum_i (x_i−\bar{x})^2/n $$

## Load in example data

In [41]:
import libpysal as lp
import geopandas as gpd
from scipy import stats
import numpy as np
guerry = lp.examples.load_example('Guerry')
guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))

In [42]:
wq = lp.weights.Queen.from_dataframe(guerry_ds)

In [43]:
wq[0]

{66: 1.0, 35: 1.0, 68: 1.0, 36: 1.0}

In [44]:
wq.transform = 'r'
wq[0]

{66: 0.25, 35: 0.25, 68: 0.25, 36: 0.25}

In [45]:
x = guerry_ds['Donatns']

print("x_i is", x[0])
print("x_j are", x[66], x[35], x[68], x[36])

In [46]:
# Calculate score
zscore_x = (x - np.mean(x))/np.std(x)
zscore_x

0    -0.336188
1     0.450441
2     0.879023
3    -0.825375
4     0.049370
        ...   
80    1.512380
81    0.454785
82    1.467288
83   -0.555029
84   -0.506214
Name: Donatns, Length: 85, dtype: float64

# Build observed local geary values

In [47]:
adj_list = wq.to_adjlist(remove_symmetric=False)
adj_list.head()

,focal,neighbor,weight
0,0,66,0.250000
1,0,35,0.250000
2,0,68,0.250000
3,0,36,0.250000
4,1,48,0.166667


In [48]:
import pandas as pd
zseries = pd.Series(zscore_x, index=wq.id_order)
zseries[0:5]

0   -0.336188
1    0.450441
2    0.879023
3   -0.825375
4    0.049370
Name: Donatns, dtype: float64

In [49]:
# Define z_i
zi = zseries.loc[adj_list.focal].values
zi[0:5]

array([-0.33618783, -0.33618783, -0.33618783, -0.33618783,  0.45044136])

In [50]:
# Define zj
zj = zseries.loc[adj_list.neighbor].values
zj[0:5]

array([-0.98050808, -0.54737594, -0.62328783, -0.76766521, -0.5709562 ])

In [51]:
(zi-zj)[0:5]

array([0.64432025, 0.21118812, 0.2871    , 0.43147738, 1.02139756])

In [15]:
#(zi-zj)**2

Multiply by spatial weights

In [13]:
#sum(list(wq.weights.values()), [])

In [14]:
# sum(list(wq.weights.values()), []) * (zi-zj)**2

In [52]:
test = sum(list(wq.weights.values()), []) * (zi-zj)**2

In [53]:
test[0:5]

array([0.10378715, 0.01115011, 0.0206066 , 0.04654318, 0.1738755 ])

In [54]:
# Create a df that uses the adjacency list focal values and the BBs counts
temp = pd.DataFrame(adj_list.focal.values, test).reset_index()
temp[0:5]

,index,0
0,0.103787,0
1,0.011150,0
2,0.020607,0
3,0.046543,0
4,0.173875,1


In [55]:
# Temporarily rename the columns
temp.columns = ['Eij', 'ID']
temp = temp.groupby(by='ID').sum()

In [56]:
temp.Eij.values[0:5]

array([0.18208704, 0.56001403, 0.97529461, 0.21590694, 0.61737256])

# Start building function

Need to add quads functionality to the statistic!

In [32]:
#%load_ext pycodestyle_magic

In [40]:
#%pycodestyle_off

In [57]:
import numpy as np
import pandas as pd
import warnings
from scipy import sparse
from scipy import stats
from sklearn.base import BaseEstimator
import libpysal as lp
from esda.crand import (
    crand as _crand_plus,
    njit as _njit,
    _prepare_univariate
)


PERMUTATIONS = 999


class Local_Geary(BaseEstimator):
    """Local Geary - Univariate"""

    def __init__(self, connectivity=None, permutations=PERMUTATIONS, n_jobs=1,
                 keep_simulations=True, seed=None):
        """
        connectivity     : scipy.sparse matrix object
                           the connectivity structure describing
                           the relationships between observed units.
                           Need not be row-standardized.
        permutations     : int
                           number of random permutations for calculation
                           of pseudo p_values
        n_jobs           : int
                           Number of cores to be used in the conditional
                           randomisation. If -1, all available cores are used.
        keep_simulations : Boolean
                           (default=True)
                           If True, the entire matrix of replications under
                           the null is stored in memory and accessible;
                           otherwise, replications are not saved
        seed             : None/int
                           Seed to ensure reproducibility of conditional
                           randomizations. Must be set here, and not outside
                           of the function, since numba does not correctly
                           interpret external seeds nor
                           numpy.random.RandomState instances.

        Attributes
        ----------
        localG          : numpy array
                          array containing the observed univariate
                          Local Geary values.
        p_sim           : numpy array
                          array containing the simulated
                          p-values for each unit.
        """

        self.connectivity = connectivity
        self.permutations = permutations
        self.n_jobs = n_jobs
        self.keep_simulations = keep_simulations
        self.seed = seed

    def fit(self, x, n_jobs=1, permutations=999):
        """
        Arguments
        ---------
        x                : numpy.ndarray
                           array containing continuous data

        Returns
        -------
        the fitted estimator.

        Notes
        -----
        Technical details and derivations can be found in :cite:`Anselin1995`.

        Examples
        --------
        Guerry data replication GeoDa tutorial
        >>> import libpysal as lp
        >>> import geopandas as gpd
        >>> guerry = lp.examples.load_example('Guerry')
        >>> guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))
        >>> w = libpysal.weights.Queen.from_dataframe(guerry_ds)
        >>> y = guerry_ds['Donatns']
        >>> lG = Local_Geary(connectivity=w).fit(y)
        >>> lG.localG[0:5]
        >>> lG.p_sim[0:5]
        """
        x = np.asarray(x).flatten()

        w = self.connectivity
        w.transform = 'r'

        self.localG = self._statistic(x, w)

        if self.permutations:
            self.p_sim, self.rlocalG = _crand_plus(
                z=(x - np.mean(x))/np.std(x),
                w=w,
                observed=self.localG,
                permutations=permutations,
                keep=True,
                n_jobs=n_jobs,
                stat_func=_local_geary
            )

        del (self.keep_simulations, self.n_jobs,
             self.permutations, self.seed, self.rlocalG,
             self.connectivity)

        return self

    @staticmethod
    def _statistic(x, w):
        # Caclulate z-scores for x
        zscore_x = (x - np.mean(x))/np.std(x)
        # Create focal (xi) and neighbor (zi) values
        adj_list = w.to_adjlist(remove_symmetric=False)
        zseries = pd.Series(zscore_x, index=w.id_order)
        zi = zseries.loc[adj_list.focal].values
        zj = zseries.loc[adj_list.neighbor].values
        # Carry out local Geary calculation
        gs = sum(list(w.weights.values()), []) * (zi-zj)**2
        # Reorganize data
        adj_list_gs = pd.DataFrame(adj_list.focal.values, gs).reset_index()
        adj_list_gs.columns = ['gs', 'ID']
        adj_list_gs = adj_list_gs.groupby(by='ID').sum()

        localG = adj_list_gs.gs.values

        return (localG)

# --------------------------------------------------------------
# Conditional Randomization Function Implementations
# --------------------------------------------------------------

# Note: does not using the scaling parameter


@_njit(fastmath=True)
def _local_geary(i, z, permuted_ids, weights_i, scaling):
    zi, zrand = _prepare_univariate(i, z, permuted_ids, weights_i)
    return (zi-zrand)**2 @ weights_i

In [58]:
functest = Local_Geary(connectivity=wq).fit(x)

In [59]:
functest.localG

array([1.82087039e-01, 5.60014026e-01, 9.75294606e-01, 2.15906938e-01,
       6.17372564e-01, 3.84450059e-02, 2.43181756e-01, 9.71802819e-01,
       4.06447101e-02, 7.24722785e-01, 6.30952854e-02, 2.42104497e-02,
       1.59496916e+01, 9.29326006e-01, 9.65188634e-01, 1.32383286e+00,
       3.31775497e-01, 2.99446505e+00, 9.43946814e-01, 2.99570159e+00,
       3.66702291e-01, 2.09592365e+00, 1.46515861e+00, 1.82118455e-01,
       3.10216680e+00, 5.43063937e-01, 5.74532559e+00, 4.79160197e-02,
       1.58993089e-01, 7.18327253e-01, 1.24297849e+00, 8.72629331e-02,
       7.52809650e-01, 4.56515485e-01, 3.86766562e-01, 1.17632604e-01,
       6.90884685e-01, 2.87206102e+00, 4.10455112e-01, 4.04349959e-01,
       1.14211758e-01, 9.59519953e-01, 3.51347976e-01, 7.30240974e-01,
       4.40370938e-01, 7.20360356e-02, 1.66241706e+00, 5.83258909e+00,
       2.30332507e-01, 4.38369688e-01, 8.41461470e-01, 1.52959486e+00,
       4.32157479e-02, 2.08325903e+00, 1.19722984e+00, 1.28169257e+00,
      

In [60]:
functest.p_sim

array([0.191, 0.061, 0.077, 0.157, 0.463, 0.002, 0.153, 0.454, 0.029,
       0.299, 0.009, 0.006, 0.172, 0.475, 0.006, 0.034, 0.005, 0.15 ,
       0.323, 0.078, 0.002, 0.169, 0.336, 0.093, 0.079, 0.335, 0.031,
       0.013, 0.024, 0.277, 0.256, 0.006, 0.498, 0.001, 0.117, 0.089,
       0.321, 0.189, 0.292, 0.157, 0.047, 0.366, 0.196, 0.494, 0.282,
       0.008, 0.208, 0.022, 0.142, 0.246, 0.292, 0.26 , 0.021, 0.06 ,
       0.433, 0.331, 0.074, 0.288, 0.03 , 0.012, 0.242, 0.413, 0.438,
       0.124, 0.43 , 0.159, 0.069, 0.11 , 0.293, 0.178, 0.087, 0.234,
       0.016, 0.252, 0.008, 0.159, 0.032, 0.046, 0.011, 0.001, 0.039,
       0.5  , 0.003, 0.343, 0.257])

In [26]:
import libpysal as lp
import geopandas as gpd
guerry = lp.examples.load_example('Guerry')
guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))
w = libpysal.weights.Queen.from_dataframe(guerry_ds)
y = guerry_ds['Donatns']
lG = Local_Geary(connectivity=w).fit(y)
lG.localG[0:5]
lG.p_sim[0:5]

array([0.19 , 0.066, 0.064, 0.177, 0.443])

## Start working on inference (note: now implemented above)

### 'New' `_crand()` engine

In [24]:
from esda.crand import (
    crand as _crand_plus,
    njit as _njit,
    _prepare_univariate
)

In [25]:
@_njit(fastmath=True)
def _local_geary(i, z, permuted_ids, weights_i, scaling):
    zi, zrand = _prepare_univariate(i, z, permuted_ids, weights_i)
    return (zi-zrand)**2 @ weights_i

In [26]:
p_sim, rlocalG = _crand_plus(z=np.array(zscore_x), w=wq, observed=np.array(functest.localG), 
            permutations=999, keep=True, n_jobs=1, 
            stat_func=_local_geary)

print(p_sim)
print(rlocalG)

[0.194 0.06  0.082 0.149 0.465 0.004 0.173 0.446 0.017 0.267 0.012 0.01
 0.152 0.474 0.006 0.031 0.003 0.154 0.316 0.06  0.002 0.157 0.303 0.092
 0.063 0.362 0.022 0.013 0.027 0.29  0.228 0.005 0.471 0.002 0.124 0.083
 0.328 0.167 0.302 0.164 0.05  0.341 0.191 0.475 0.289 0.003 0.204 0.023
 0.129 0.252 0.291 0.244 0.031 0.085 0.421 0.364 0.08  0.312 0.021 0.021
 0.228 0.358 0.492 0.107 0.422 0.138 0.09  0.095 0.307 0.191 0.12  0.219
 0.013 0.271 0.005 0.175 0.038 0.041 0.008 0.001 0.033 0.45  0.002 0.322
 0.262]
[[2.05263347 0.26799261 0.18726531 ... 5.59551063 0.07625188 0.61707767]
 [1.00043766 0.92158741 1.22897191 ... 3.43241739 1.20261723 0.48587636]
 [1.29903936 1.64818981 2.00888232 ... 3.41923814 2.32060801 1.05184552]
 ...
 [2.87201109 2.94999783 1.95852471 ... 3.24047778 3.3677634  2.2651586 ]
 [0.41795495 0.80604515 1.31935463 ... 0.87928535 0.35429401 2.27589509]
 [0.4498144  0.90007996 0.96779908 ... 1.00388175 0.39270281 2.615005  ]]


# Multivariate Local Geary

$$ c_i = \sum_{h=1}^m \sum_j w_{ij} (x_{hi} - x_{hj})^2 $$

Load in the sample data

In [27]:
import libpysal as lp
import geopandas as gpd
from scipy import stats
import pandas as pd
guerry = lp.examples.load_example('Guerry')
guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))

In [28]:
wq = lp.weights.Queen.from_dataframe(guerry_ds)

In [29]:
x = guerry_ds['Donatns']
y = guerry_ds['Suicids']

In [30]:
variables = [x,y]

Standardize each variable

In [31]:
from scipy import stats
zseries = [stats.zscore(i) for i in variables]

Build the adj lists

In [32]:
adj_list = wq.to_adjlist(remove_symmetric=False)

In [33]:
# The zseries
zseries = [pd.Series(i, index=wq.id_order) for i in zseries]

In [61]:
zseries[0:10]

[0    -0.336188
 1     0.450441
 2     0.879023
 3    -0.825375
 4     0.049370
         ...   
 80    1.512380
 81    0.454785
 82    1.467288
 83   -0.555029
 84   -0.506214
 Length: 85, dtype: float64,
 0    -0.047195
 1    -0.756433
 2     2.478379
 3    -0.711499
 4    -0.649766
         ...   
 80    1.004270
 81   -0.468369
 82   -0.096441
 83   -0.111387
 84   -0.757774
 Length: 85, dtype: float64]

In [35]:
# The focal values
focal = [zseries[i].loc[adj_list.focal].values for
         i in range(len(variables))]
# The neighbor values
neighbor = [zseries[i].loc[adj_list.neighbor].values for
            i in range(len(variables))]

In [36]:
np.array(focal)[0][0]

-0.3361878263899247

In [37]:
np.array(neighbor)[0][0]

-0.9805080804028404

In [38]:
(np.array(focal)[0][0] - np.array(neighbor)[0][0])**2

0.4151485897312682

In [39]:
temp = (np.array(focal) - np.array(neighbor))**2
temp[0][0]

0.4151485897312682

In [40]:
gs = sum(list(wq.weights.values()), []) * (np.array(focal) - np.array(neighbor))**2

In [41]:
temp = pd.DataFrame(gs).T

In [42]:
temp['ID'] = adj_list.focal.values

In [43]:
adj_list_gs = temp.groupby(by='ID').sum()
adj_list_gs.head()

,0,1
ID,,
0,0.728348,0.502200
1,3.360084,0.282630
2,5.851768,29.604873
3,0.863628,0.121489
4,1.852118,0.475642


In [44]:
k = len(variables)
k

2

In [45]:
adj_list_gs.sum(axis=1)/k

ID
0      0.615274
1      1.821357
2     17.728320
3      0.492558
4      1.163880
        ...    
80     6.629720
81     3.154587
82     3.872022
83     4.307689
84     1.080905
Length: 85, dtype: float64

In [46]:
import numpy as np
import pandas as pd
import warnings
from scipy import sparse
from scipy import stats
from sklearn.base import BaseEstimator
import libpysal as lp

PERMUTATIONS=999

class Local_Geary_MV(BaseEstimator):
    """Local Geary - Multivariate"""

    def __init__(self, connectivity=None, permutations=PERMUTATIONS, n_jobs=1, 
                 keep_simulations=True, seed=None):
        """
        connectivity     : scipy.sparse matrix object
                           the connectivity structure describing
                           the relationships between observed units.
                           Need not be row-standardized.
        permutations     : int
                           number of random permutations for calculation of pseudo
                           p_values
        n_jobs           : int
                           Number of cores to be used in the conditional randomisation. If -1,
                           all available cores are used.    
        keep_simulations : Boolean
                           (default=True)
                           If True, the entire matrix of replications under the null 
                           is stored in memory and accessible; otherwise, replications 
                           are not saved
        seed             : None/int
                           Seed to ensure reproducibility of conditional randomizations. 
                           Must be set here, and not outside of the function, since numba 
                           does not correctly interpret external seeds 
                           nor numpy.random.RandomState instances.  
                           
        Attributes
        ----------
        localG          : numpy array
                          array containing the observed multivariate
                          Local Geary values.
        p_sim           : numpy array
                          array containing the simulated
                          p-values for each unit.
        """

        self.connectivity = connectivity
        self.permutations = permutations
        self.n_jobs = n_jobs
        self.keep_simulations = keep_simulations
        self.seed = seed

    def fit(self, variables, n_jobs=1, permutations=999):
        """
        Arguments
        ---------
        variables        : numpy.ndarray
                           array containing continuous data

        Returns
        -------
        the fitted estimator.

        Notes
        -----
        Technical details and derivations can be found in :cite:`Anselin1995`.

        Examples
        --------
        Guerry data replication GeoDa tutorial
        >>> import libpysal
        >>> import geopandas as gpd
        >>> guerry = lp.examples.load_example('Guerry')
        >>> guerry_ds = gpd.read_file(guerry.get_path('Guerry.shp'))
        >>> w = libpysal.weights.Queen.from_dataframe(guerry_ds)
        """
        self.variables = np.array(variables, dtype='float')

        w = self.connectivity
        w.transform = 'r'

        self.localG = self._statistic(variables, w)

        if permutations:
            pass

        return self

    @staticmethod
    def _statistic(variables, w):
        # Caclulate z-scores for input variables
        zseries = [stats.zscore(i) for i in variables]
        # Define denominator adjustment
        k = len(variables)
        # Create focal and neighbor values
        adj_list = w.to_adjlist(remove_symmetric=False)
        zseries = [pd.Series(i, index=wq.id_order) for i in zseries]
        focal = [zseries[i].loc[adj_list.focal].values for
                 i in range(len(variables))]
        neighbor = [zseries[i].loc[adj_list.neighbor].values for
                    i in range(len(variables))]
        # Carry out local Geary calculation
        gs = sum(list(wq.weights.values()), []) * \
        (np.array(focal) - np.array(neighbor))**2
        # Reorganize data
        temp = pd.DataFrame(gs).T
        temp['ID'] = adj_list.focal.values
        adj_list_gs = temp.groupby(by='ID').sum()
        localG = adj_list_gs.sum(axis=1)/k
        
        return (localG)

# --------------------------------------------------------------
# Conditional Randomization Function Implementations
# --------------------------------------------------------------

# Note: does not using the scaling parameter

@_njit(fastmath=True)
def _local_geary(i, z, permuted_ids, weights_i, scaling):
    zi, zrand = _prepare_univariate(i, z, permuted_ids, weights_i)
    return (zi-zrand)**2 @ weights_i    

In [47]:
functest = Local_Geary_MV(connectivity=wq).fit([x,y])
functest.localG

ID
0     0.153819
1     0.303560
2     2.954720
3     0.123140
4     0.387960
        ...   
80    1.657430
81    0.525764
82    0.645337
83    0.717948
84    0.216181
Length: 85, dtype: float64

## Working on inference

In [48]:
from esda.crand import (
    crand as _crand_plus,
    njit as _njit,
    _prepare_univariate
)

In [49]:
@_njit(fastmath=True)
def _local_geary(i, z, permuted_ids, weights_i, scaling):
    zi, zrand = _prepare_univariate(i, z, permuted_ids, weights_i)
    return (zi-zrand)**2 @ weights_i

In [50]:
p_sim, rlocalG = _crand_plus(z=np.array(x, dtype='float'), w=wq, observed=np.array(functest.localG), 
            permutations=999, keep=True, n_jobs=1, 
            stat_func=_local_geary)

print(p_sim)
print(rlocalG)

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001]
[[9.80030950e+06 4.96861950e+06 4.33885925e+06 ... 3.41027162e+07
  6.95303925e+06 1.46068250e+05]
 [2.65690365e+07 2.30316072e+07 2.08195587e+07 ... 2.20637503e+07
  3.04229432e+07 1.78539172e+07]
 [4.40311618e+07 3.90267565e+07 4.25120173e+07 ... 2.71152863e+07
  5.63719805e+07 3.11692798e+07]
 ...
 [1.03346886e+08 4.92207678e+07 7.71749292e+07 ... 4.82089275e+07
  8.95876458e+07 9.53348528e+07]
 [1.14138733e+06 1.00308852e+08 1.25228872e+07 ... 3.23375975e+07
  1.07198458e+

In [51]:
x_zscore = stats.zscore(x)
y_zscore = stats.zscore(y)

In [52]:
result = zip(x_zscore, y_zscore)
result = list(result)
result = np.array(result)
result[0:5]

array([[-0.33618783, -0.04719523],
       [ 0.45044136, -0.75643306],
       [ 0.87902293,  2.47837872],
       [-0.82537479, -0.71149891],
       [ 0.0493701 , -0.64976635]])

In [53]:
permutations = 999
# Get length based on first variable
n = len(result)
localGs = np.zeros((n, permutations))
n_1 = n - 1
prange = list(range(permutations))
k = wq.max_neighbors + 1
nn = n - 1
rids = np.array([np.random.permutation(nn)[0:k] for i in prange])
ids = np.arange(wq.n)
ido = wq.id_order
w = [wq.weights[ido[i]] for i in ids]
wc = [wq.cardinalities[ido[i]] for i in ids]


Note: below **are** z-score standardized

In [54]:
wq.weights[0]

[0.25, 0.25, 0.25, 0.25]

In [55]:
for i in range(wq.n):
    idsi = ids[ids != i]
    np.random.shuffle(idsi)
    tmp = result[idsi[rids[:, 0:wc[i]]]]
    # define zi?
    zi = result[i][0]
    # define zrand?
    zrand = tmp[i][:,1]
    # Subtract and square?
    temp = (zi - zrand)**2
    # Multiply by weights?
    localGs[i] = (temp * wq.weights[i]).sum(0)
    #joins[i] = result[i] * (w[i] * tmp).sum(1)
    #localGs[i] = ((result[i,0] - tmp2)**2 * wq.weights[i][0]).sum(0)
rlocalGs = localGs

In [56]:
rlocalGs

array([[3.9717587 , 3.9717587 , 3.9717587 , ..., 3.9717587 , 3.9717587 ,
        3.9717587 ],
       [1.74989996, 1.74989996, 1.74989996, ..., 1.74989996, 1.74989996,
        1.74989996],
       [1.84848395, 1.84848395, 1.84848395, ..., 1.84848395, 1.84848395,
        1.84848395],
       ...,
       [2.93835176, 2.93835176, 2.93835176, ..., 2.93835176, 2.93835176,
        2.93835176],
       [0.10131788, 0.10131788, 0.10131788, ..., 0.10131788, 0.10131788,
        0.10131788],
       [0.40014853, 0.40014853, 0.40014853, ..., 0.40014853, 0.40014853,
        0.40014853]])

In [57]:
np.transpose(rlocalGs)

array([[3.9717587 , 1.74989996, 1.84848395, ..., 2.93835176, 0.10131788,
        0.40014853],
       [3.9717587 , 1.74989996, 1.84848395, ..., 2.93835176, 0.10131788,
        0.40014853],
       [3.9717587 , 1.74989996, 1.84848395, ..., 2.93835176, 0.10131788,
        0.40014853],
       ...,
       [3.9717587 , 1.74989996, 1.84848395, ..., 2.93835176, 0.10131788,
        0.40014853],
       [3.9717587 , 1.74989996, 1.84848395, ..., 2.93835176, 0.10131788,
        0.40014853],
       [3.9717587 , 1.74989996, 1.84848395, ..., 2.93835176, 0.10131788,
        0.40014853]])

In [58]:
sim = np.transpose(rlocalGs)
print(sim[0])
print(sim[1])

[ 3.9717587   1.74989996  1.84848395  0.16259905  0.60815684  0.69461573
  7.3435836   0.55864019  0.22311996  0.96740117  0.38905425  1.94533832
 15.77345769  0.5066816   3.15569397  3.25510059  1.48485159  2.60949699
  0.62161498  1.57624453  1.70909514  0.06951547  9.30119783  3.11385176
  2.3162792   1.05999882 15.53313673  0.39441147  1.22141286  1.5902047
  2.0403228   3.58843212  0.49414432  2.25973022  0.63619647  0.37923618
  2.62674149  2.57162957  0.29228435  3.4159719   0.82509152  1.24728002
  1.80149777  1.7808194   0.81275629  6.83743352  0.07893041  1.06556625
  1.1366002   0.942019    6.11250823  0.09168751  0.56992065  4.13389421
  3.16959962  1.06396666  4.56573381  0.30289781  0.82284309  1.29017296
  1.91483863  0.19379776  2.48910052  2.08155351  3.8453653   0.33514834
  1.27133425  3.18939375  1.34585107  0.18252789  0.1738293   0.71667069
  1.37785345  0.71270744  5.26570802  0.16598848  3.17905464  3.64248918
  0.58383773  1.26728078  2.39828116  2.27259217  2.

In [59]:
sim[0]

array([ 3.9717587 ,  1.74989996,  1.84848395,  0.16259905,  0.60815684,
        0.69461573,  7.3435836 ,  0.55864019,  0.22311996,  0.96740117,
        0.38905425,  1.94533832, 15.77345769,  0.5066816 ,  3.15569397,
        3.25510059,  1.48485159,  2.60949699,  0.62161498,  1.57624453,
        1.70909514,  0.06951547,  9.30119783,  3.11385176,  2.3162792 ,
        1.05999882, 15.53313673,  0.39441147,  1.22141286,  1.5902047 ,
        2.0403228 ,  3.58843212,  0.49414432,  2.25973022,  0.63619647,
        0.37923618,  2.62674149,  2.57162957,  0.29228435,  3.4159719 ,
        0.82509152,  1.24728002,  1.80149777,  1.7808194 ,  0.81275629,
        6.83743352,  0.07893041,  1.06556625,  1.1366002 ,  0.942019  ,
        6.11250823,  0.09168751,  0.56992065,  4.13389421,  3.16959962,
        1.06396666,  4.56573381,  0.30289781,  0.82284309,  1.29017296,
        1.91483863,  0.19379776,  2.48910052,  2.08155351,  3.8453653 ,
        0.33514834,  1.27133425,  3.18939375,  1.34585107,  0.18

In [60]:
larger[False]

NameError: name 'larger' is not defined

In [ ]:
above = sim[0] >= functest.localG
larger = above.sum(0)
low_extreme = (permutations - larger) < larger
larger[low_extreme] = permutations - larger[low_extreme]
p_sim = (larger + 1.0) / (permutations + 1.0)
p_sim

In [ ]:
temp2